In [ ]:
# Mount google drive at /content/drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Set seeds
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import StandardScaler
import json
import pandas as pd
np.random.seed(42)
tf.random.set_seed(42)

In [ ]:
# Fetching PCP properties of druggable and non-druggable proteins
data_file_path = "/content/drive/MyDrive/protein_props.json"
druggable_proteins_file_path = "/content/drive/MyDrive/druggable_proteins.txt"
approved_druggable_proteins_file_path = "/content/drive/MyDrive/approved_druggable_proteins.txt"

with open(data_file_path, 'r') as f:
    protein_data = json.load(f)

print("Total number of uniprot human verified proteins:", len(protein_data))

with open(druggable_proteins_file_path, 'r') as f:
    druggable_proteins = f.read().splitlines()

with open(approved_druggable_proteins_file_path, 'r') as f:
    approved_druggable_proteins = f.read().splitlines()

print("Number of druggable proteins:", len(druggable_proteins))
print("Number of approved druggable proteins:", len(approved_druggable_proteins))

properties = (pd.read_json("/content/drive/MyDrive/protein_props.json")).transpose()
is_druggable = [1 if i in druggable_proteins else 0 for i in properties.index]
is_approved_druggable = [1 if i in approved_druggable_proteins else 0 for i in properties.index]

properties["is_druggable"] = is_druggable
properties["is_approved_druggable"] = is_approved_druggable

PCP_properties = properties.copy()
amino_acids = 'ACDEFGHIKLMNPQRSTVWY'
amino_acid_percent = {i:[] for i in amino_acids}
for i in PCP_properties['Amino Acid Percent']:
  for aa in amino_acids:
    amino_acid_percent[aa].append(i[aa])
for aa in amino_acids:
  PCP_properties = pd.concat([PCP_properties, pd.Series(amino_acid_percent[aa], index = PCP_properties.index, name = f"Amino Acid Percent {aa}")], axis = 1)

PCP_properties[f"Molar Extinction Coefficient 1"] = pd.Series([x[0] for x in PCP_properties['Molar Extinction Coefficient']], index = PCP_properties.index)
PCP_properties[f"Molar Extinction Coefficient 2"] = pd.Series([x[1] for x in PCP_properties['Molar Extinction Coefficient']], index = PCP_properties.index)

PCP_properties[f"Secondary Structure helix"] = pd.Series([x[0] for x in PCP_properties['Secondary Structure']], index = PCP_properties.index)
PCP_properties[f"Secondary Structure turn"] = pd.Series([x[1] for x in PCP_properties['Secondary Structure']], index = PCP_properties.index)
PCP_properties[f"Secondary Structure sheet"] = pd.Series([x[2] for x in PCP_properties['Secondary Structure']], index = PCP_properties.index)

PCP_properties.drop(columns = ['Amino Acid Count','Amino Acid Percent',"Molar Extinction Coefficient","Flexibility","Secondary Structure",'Sequence'], inplace = True)
PCP_properties['Sequence Length'] = PCP_properties['Sequence Length'].astype(int)
PCP_properties[['Molecular Weight', 'GRAVY', 'Isoelectric Point', 'Instability Index', 'Aromaticity', 'Charge at 7']] = PCP_properties[['Molecular Weight', 'GRAVY', 'Isoelectric Point', 'Instability Index', 'Aromaticity', 'Charge at 7']].astype(float)

with open("/content/drive/MyDrive/BDDF_Research/gdpc_encodings.json", 'r') as file:
    data = json.load(file)
gpdc_encodings = pd.DataFrame(data).transpose()

ppi = pd.read_json("/content/drive/MyDrive/ppi.json").transpose()
ppi_network = pd.read_csv("/content/drive/MyDrive/BDDF_Research/ppi_network_properties.csv")
ppi_network.index = ppi_network['Unnamed: 0']
ppi_network.drop(columns = ['Unnamed: 0'], inplace = True)
ppi = pd.concat([ppi, ppi_network], axis = 1)

glycolisation = pd.read_csv("/content/drive/MyDrive/glycosylation.csv")
glycolisation.index = glycolisation['Unnamed: 0']
glycolisation.drop(columns = ['Unnamed: 0'], inplace = True)
ptm = pd.read_csv("/content/drive/MyDrive/PTM_counts.csv")
ptm.index = ptm["Unnamed: 0"]
ptm.drop(columns = ['Unnamed: 0'], inplace = True)
ptm_counts = pd.concat([ptm, glycolisation], axis = 1)

with open("/content/drive/MyDrive/subcellular_locations2.json", 'r') as file:
    data = json.load(file)
unique_groups = set()
for entry in data.values():
    if "general" in entry:
        for general_entry in entry["general"]:
            if "group" in general_entry: unique_groups.add(general_entry["group"])

unique_groups_list = list(unique_groups)

rows = []
for protein_id in PCP_properties.index:
    row = {group: 0 for group in unique_groups_list}
    if protein_id in data:
        for entry in data[protein_id].get("general", []):
            if "group" in entry and entry["group"] in unique_groups:
                row[entry["group"]] = 1
    row["protein_id"] = protein_id
    rows.append(row)

subcellular_data = pd.DataFrame(rows).set_index("protein_id")

domains = pd.read_csv("/content/drive/MyDrive/BDDF_Research/data_top20_updated.csv")
domains.index = domains['Unnamed: 0']
domains.drop(columns = ['Unnamed: 0'], inplace = True)

flexibility = pd.read_csv("/content/drive/MyDrive/BDDF_Research/flexibility_properties.csv")
flexibility.index = flexibility['Unnamed: 0']
flexibility.drop(columns = ['Unnamed: 0'], inplace = True)

latent_data = pd.read_csv("/content/drive/MyDrive/BDDF_Research/latent_values.csv").transpose()
latent_data.columns = [f"Latent_Value_{i+1}" for i in latent_data.columns]
final_data = pd.concat([PCP_properties,gpdc_encodings, ptm_counts, ppi, subcellular_data, domains, flexibility, latent_data], axis = 1).dropna()
features_list = final_data.columns
features_list = features_list.drop(['is_druggable','is_approved_druggable'])
features_list = list(features_list)
print(features_list)
print(len(features_list))

Total number of uniprot human verified proteins: 20434
Number of druggable proteins: 3345
Number of approved druggable proteins: 2652
['Sequence Length', 'Molecular Weight', 'GRAVY', 'Isoelectric Point', 'Instability Index', 'Aromaticity', 'Charge at 7', 'Amino Acid Percent A', 'Amino Acid Percent C', 'Amino Acid Percent D', 'Amino Acid Percent E', 'Amino Acid Percent F', 'Amino Acid Percent G', 'Amino Acid Percent H', 'Amino Acid Percent I', 'Amino Acid Percent K', 'Amino Acid Percent L', 'Amino Acid Percent M', 'Amino Acid Percent N', 'Amino Acid Percent P', 'Amino Acid Percent Q', 'Amino Acid Percent R', 'Amino Acid Percent S', 'Amino Acid Percent T', 'Amino Acid Percent V', 'Amino Acid Percent W', 'Amino Acid Percent Y', 'Molar Extinction Coefficient 1', 'Molar Extinction Coefficient 2', 'Secondary Structure helix', 'Secondary Structure turn', 'Secondary Structure sheet', 'aliphatic_aliphatic', 'aliphatic_positive', 'aliphatic_negative', 'aliphatic_uncharged', 'aliphatic_aromatic',

In [ ]:
#for splitting of data
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from imblearn.over_sampling import ADASYN, SMOTE

def get_data(x_sample, y_sample):
  return np.array(x_sample), np.array(y_sample)

def data_splitting(x_sample, y_sample, mode="default", scaler="none", class_size=600):
  druggable_indices = (y_sample == 1)  # Assuming 1 represents druggable
  non_druggable_indices = (y_sample == 0)  # Assuming 0 represents non-druggable

  druggable_X = x_sample[druggable_indices]
  druggable_y = y_sample[druggable_indices]

  non_druggable_X = x_sample[non_druggable_indices]
  non_druggable_y = y_sample[non_druggable_indices]

  druggable_X_remaining, druggable_X_test, druggable_y_remaining, druggable_y_test = train_test_split(druggable_X, druggable_y, test_size=class_size, random_state=123)
  non_druggable_X_remaining, non_druggable_X_test, non_druggable_y_remaining, non_druggable_y_test = train_test_split(non_druggable_X, non_druggable_y, test_size= class_size, random_state=123)

  X_test = pd.concat((druggable_X_test, non_druggable_X_test))
  y_test = pd.concat((druggable_y_test, non_druggable_y_test))
  X_train = pd.concat((druggable_X_remaining, non_druggable_X_remaining))
  y_train = pd.concat((druggable_y_remaining, non_druggable_y_remaining))
  X_train, y_train = shuffle(X_train, y_train, random_state=123)
  if mode == "default":
    pass
  elif mode == "adasyn":
    ada = ADASYN(random_state=42)
    X_train, y_train = ada.fit_resample(X_train, y_train)
  elif mode == "smote":
    smt = SMOTE(random_state=42)
    X_train, y_train = smt.fit_resample(X_train, y_train)

  if scaler == "std":
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)
  elif scaler == "minmax":
    scaler = MinMaxScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)
  elif scaler == "none":
    pass

  return X_train, X_test, y_train, y_test

In [ ]:
X_train, X_test, y_train, y_test = data_splitting(final_data[features_list], final_data["is_druggable"],mode = "scaler", class_size=600)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((19073, 183), (1200, 183), (19073,), (1200,))

In [ ]:

X_train_druggable = X_train[y_train == 1]
X_train_non_druggable = X_train[y_train == 0]

X_train_non_druggable_partitions = np.array_split(X_train_non_druggable, int(len(X_train_non_druggable)/len(X_train_druggable)))
print(f"Splitting into {len(X_train_non_druggable_partitions)} partitions")

Splitting into 6 partitions


In [ ]:

import xgboost as xgb

xgb_models = []
for partition in X_train_non_druggable_partitions:
  X_combined = np.concatenate((X_train_druggable, partition))
  y_combined = np.concatenate((np.ones(len(X_train_druggable)), np.zeros(len(partition))))
  xgb_model = xgb.XGBClassifier(objective='binary:logistic', random_state=42)
  xgb_model.fit(X_combined, y_combined)
  xgb_models.append(xgb_model)

In [ ]:
y_preds = []
for model in xgb_models:
  y_pred = model.predict(X_test)
  y_preds.append(y_pred)

majority_preds = np.mean(y_preds, axis=0)
majority_preds = np.round(majority_preds)

In [ ]:

from sklearn.metrics import accuracy_score
accuracy_metrics = {}
for i, y_pred in enumerate(y_preds):
  accuracy_metrics[f"partition_{i}"]={
      "accuracy_total": accuracy_score(y_test, y_pred),
      "accuracy_druggable": accuracy_score(y_test[y_test == 1], y_pred[y_test == 1]),
      "accuracy_non_druggable": accuracy_score(y_test[y_test == 0], y_pred[y_test == 0]),
  }

accuracy_metrics["majority"] = {
    "accuracy_total": accuracy_score(y_test, majority_preds),
    "accuracy_druggable": accuracy_score(y_test[y_test == 1], majority_preds[y_test == 1]),
    "accuracy_non_druggable": accuracy_score(y_test[y_test == 0], majority_preds[y_test == 0]),
}

df = pd.DataFrame(accuracy_metrics).transpose()
df


,accuracy_total,accuracy_druggable,accuracy_non_druggable
partition_0,0.751667,0.760000,0.743333
partition_1,0.743333,0.738333,0.748333
partition_2,0.730000,0.743333,0.716667
partition_3,0.746667,0.750000,0.743333
partition_4,0.759167,0.775000,0.743333
partition_5,0.745833,0.748333,0.743333
majority,0.764167,0.741667,0.786667


In [ ]:
!pip install sklearn-genetic-opt

In [ ]:
#Feature Selection with partition data using genetic algortithm

from sklearn_genetic import GAFeatureSelectionCV
from sklearn_genetic.plots import plot_fitness_evolution

preds = []
estimators = []
features = []

for i, partition in enumerate(X_train_non_druggable_partitions):
    X_combined = np.concatenate((X_train_druggable, partition))
    y_combined = np.concatenate((np.ones(len(X_train_druggable)), np.zeros(len(partition))))

    clf = xgb.XGBClassifier(objective='binary:logistic', random_state=42)

    evolved_estimator = GAFeatureSelectionCV(
        estimator=clf,
        cv=3,
        scoring="accuracy",
        population_size=40,
        generations=10,
        n_jobs=-1,
        verbose=True,
        keep_top_k=2,
        elitism=True,
    )

    evolved_estimator.fit(X_combined, y_combined)
    estimators.append(evolved_estimator)
    selected_features = evolved_estimator.support_
    features.append(selected_features)

    print(f"Selected features for partition {i}: {selected_features}")

    y_predict_ga = evolved_estimator.predict(X_test)
    preds.append(y_predict_ga)

/usr/local/lib/python3.10/dist-packages/deap/creator.py:185: RuntimeWarning: A class named 'FitnessMax' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "
/usr/local/lib/python3.10/dist-packages/deap/creator.py:185: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "
/usr/local/lib/python3.10/dist-packages/joblib/externals/loky/backend/fork_exec.py:38: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid = os.fork()


gen	nevals	fitness 	fitness_std	fitness_max	fitness_min
0  	40    	0.717321	0.0106038  	0.73775    	0.689854   
1  	80    	0.727575	0.00708533 	0.744174   	0.714263   
2  	80    	0.730516	0.00762782 	0.744174   	0.716098   
3  	80    	0.729974	0.00985326 	0.744174   	0.706921   
4  	80    	0.73303 	0.00820735 	0.744174   	0.705636   
5  	80    	0.732448	0.00781236 	0.744174   	0.714813   
6  	80    	0.733888	0.00838703 	0.746744   	0.706923   
7  	80    	0.735283	0.00696207 	0.746744   	0.709489   
8  	80    	0.733774	0.00975842 	0.746744   	0.709492   
9  	80    	0.734572	0.0103644  	0.748212   	0.712426   
10 	80    	0.732177	0.0100955  	0.746744   	0.707287   
Selected features for partition 0: [False  True  True False False  True  True  True False False  True False
 False  True  True  True  True  True  True False  True False  True  True
  True  True  True False  True  True  True  True  True  True False  True
  True  True False False False  True  True  True False False  True False
 

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:458: UserWarning: X has feature names, but GAFeatureSelectionCV was fitted without feature names
  warnings.warn(


gen	nevals	fitness 	fitness_std	fitness_max	fitness_min
0  	40    	0.721198	0.0101481  	0.740506   	0.697562   
1  	80    	0.729099	0.00570264 	0.740506   	0.71426    
2  	80    	0.729236	0.00622447 	0.739218   	0.716281   
3  	80    	0.732076	0.00442962 	0.739218   	0.718298   
4  	80    	0.732411	0.00584578 	0.74289    	0.715913   
5  	80    	0.731306	0.00798797 	0.74289    	0.703251   
6  	80    	0.733448	0.00640764 	0.74289    	0.711509   
7  	80    	0.731957	0.00758159 	0.74289    	0.706735   
8  	80    	0.733324	0.00655459 	0.74289    	0.713895   
9  	80    	0.733563	0.00727104 	0.74289    	0.708756   
10 	80    	0.735576	0.00809962 	0.747847   	0.711689   
Selected features for partition 1: [ True False  True  True False  True False False  True  True False  True
  True False  True  True  True  True  True  True  True False  True False
 False  True False False False  True False  True  True False  True  True
 False False  True  True False  True  True  True  True False  True False
 

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:458: UserWarning: X has feature names, but GAFeatureSelectionCV was fitted without feature names
  warnings.warn(


...

gen	nevals	fitness 	fitness_std	fitness_max	fitness_min
0  	40    	0.719339	0.012582   	0.745641   	0.693707   
1  	80    	0.733006	0.00761618 	0.745641   	0.71004    
2  	80    	0.733718	0.00711429 	0.74601    	0.716463   
3  	80    	0.733144	0.00877388 	0.74601    	0.710039   
4  	80    	0.734278	0.00651955 	0.744358   	0.710039   
5  	80    	0.73636 	0.00661105 	0.744358   	0.708205   
6  	80    	0.734759	0.00721324 	0.743623   	0.714995   
7  	80    	0.735773	0.00765222 	0.74399    	0.708936   
8  	80    	0.733364	0.00915319 	0.743439   	0.708936   
9  	80    	0.736576	0.00665053 	0.743439   	0.716648   
10 	80    	0.733488	0.00831478 	0.743439   	0.710223   
Selected features for partition 3: [False  True False False  True False  True  True  True  True False  True
  True  True  True  True  True  True False  True False  True  True  True
  True False False  True False False False False False  True  True  True
 False False False False  True  True  True False  True  True  True False
 

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:458: UserWarning: X has feature names, but GAFeatureSelectionCV was fitted without feature names
  warnings.warn(


gen	nevals	fitness 	fitness_std	fitness_max	fitness_min
0  	40    	0.724885	0.0104359  	0.745273   	0.703801   
1  	80    	0.732203	0.00739599 	0.74766    	0.716645   
2  	80    	0.736492	0.00709854 	0.746741   	0.711323   
3  	80    	0.737447	0.00735068 	0.746741   	0.71371    
4  	80    	0.737938	0.00772151 	0.747843   	0.711322   
5  	80    	0.738493	0.00798919 	0.747843   	0.716094   
6  	80    	0.737484	0.00770315 	0.751514   	0.719032   
7  	80    	0.73964 	0.00636428 	0.751514   	0.723619   
8  	80    	0.739241	0.00844636 	0.751514   	0.714077   
9  	80    	0.7397  	0.00795977 	0.751514   	0.703433   
10 	80    	0.741837	0.0065171  	0.751514   	0.718847   
Selected features for partition 4: [False  True False  True  True False  True  True False  True  True False
  True False False  True  True  True  True False  True False  True False
  True  True False  True  True False False  True False  True  True False
 False  True  True False False  True False  True  True  True False  True
 

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:458: UserWarning: X has feature names, but GAFeatureSelectionCV was fitted without feature names
  warnings.warn(


gen	nevals	fitness 	fitness_std	fitness_max	fitness_min
0  	40    	0.720338	0.0103634  	0.742658   	0.700257   
1  	80    	0.732636	0.00719179 	0.743943   	0.716043   
2  	80    	0.73332 	0.00849788 	0.743943   	0.707232   
3  	80    	0.734022	0.0098647  	0.743943   	0.705396   
4  	80    	0.733044	0.0112232  	0.743576   	0.703928   
5  	80    	0.73678 	0.00880086 	0.747981   	0.707966   
6  	80    	0.735444	0.00927066 	0.747981   	0.706681   


In [ ]:
import matplotlib
from matplotlib.pyplot import plt

plot = plot_fitness_evolution(estimators[0], metric="fitness")
plt.show()

In [ ]:
plot = plot_fitness_evolution(estimators[1], metric="fitness")
plt.show()

In [ ]:
plot = plot_fitness_evolution(estimators[2], metric="fitness")
plt.show()

In [ ]:
plot = plot_fitness_evolution(estimators[3], metric="fitness")
plt.show()

In [ ]:
plot = plot_fitness_evolution(estimators[4], metric="fitness")
plt.show()

In [ ]:
import pandas as pd


majority_preds = np.mean(preds, axis=0)
majority_preds = np.round(majority_preds)


from sklearn.metrics import accuracy_score
accuracy_metrics = {}
for i, y_pred in enumerate(y_preds):
  accuracy_metrics[f"partition_{i}"]={
      "accuracy_total": accuracy_score(y_test, y_pred),
      "accuracy_druggable": accuracy_score(y_test[y_test == 1], y_pred[y_test == 1]),
      "accuracy_non_druggable": accuracy_score(y_test[y_test == 0], y_pred[y_test == 0]),
  }

accuracy_metrics["majority"] = {
    "accuracy_total": accuracy_score(y_test, majority_preds),
    "accuracy_druggable": accuracy_score(y_test[y_test == 1], majority_preds[y_test == 1]),
    "accuracy_non_druggable": accuracy_score(y_test[y_test == 0], majority_preds[y_test == 0]),
}

df = pd.DataFrame(accuracy_metrics).transpose()
df

In [ ]:
import xgboost as xgb
estimator_0_features = np.array([False, True, True, False, False, True, True, True, False, False, True, False,
 False, True, True, True, True, True, True, False, True, False, True, True,
 True, True, True, False, True, True, True, True, True, True, False, True,
 True, True, False, False, False, True, True, True, False, False, True, False,
 False, False, True, True, True, True, True, True, True, True, True, True,
 False, False, True, False, True, False, True, True, True, False, True, True,
 True, True, True, False, False, True, True, True, False, False, True, False,
 False, False, True, True, False, True, True, True, True, True, True, False,
 False, True, True, False, True, False, False, False, True, True, True, True,
 True, True, True, False, True, False, False, True, True, False, True, False,
 True, False, False, False, False, False, False, True, False, True, True, True,
 False, False, False, False, True, False, False, True, True, False, True, True,
 True, False, True, True, False, False, False, True, False, True, False, False,
 True, False, True, True, False, False, False, False, True, False, False, True,
 True, False, False, False, True, False, True, True, True, False, False, False,
 True, True, True])
estimator_1_features = np.array([True, False, True, True, False, True, False, False, True, True, False, True,
True, False, True, True, True, True, True, True, True, False, True, False,
False, True, False, False, False, True, False, True, True, False, True, True,
False, False, True, True, False, True, True, True, True, False, True, False,
True, True, False, False, True, False, False, True, False, True, True, True,
True, False, True, False, False, False, True, True, False, True, True, False,
False, True, True, True, False, False, True, False, False, False, True, False,
False, False, False, True, True, False, False, False, False, False, True, False,
False, False, True, False, False, False, False, False, True, False, True, False,
True, True, False, True, True, False, True, True, True, False, True, True,
False, True, True, True, True, True, False, False, False, False, True, False,
True, True, True, False, False, True, False, True, False, True, False, True,
True, False, False, False, False, True, True, True, False, False, False, True,
True, False, True, True, False, True, True, False, False, False, True, True,
False, False, False, False, False, True, True, True, False, False, False, True,
False, True, True]
)
estimator_2_features = np.array([True, False, False, False, True, True, True, True, True, False, False, True,
False, False, True, False, True, True, False, False, False, True, True, False,
True, False, False, True, True, True, True, True, False, True, True, True,
True, False, True, False, True, False, False, False, False, False, True, True,
True, True, False, False, False, False, True, True, False, False, False, True,
False, True, True, True, True, False, False, False, True, False, False, False,
False, True, True, True, False, True, True, False, True, True, True, True,
False, False, False, False, True, True, False, True, True, False, False, True,
True, False, False, True, False, False, True, True, True, False, True, True,
False, False, True, True, True, False, False, True, False, False, True, False,
False, False, False, False, False, False, False, True, True, False, False, True,
False, False, False, False, True, True, False, True, False, False, False, True,
False, True, False, True, True, True, True, True, False, True, True, False,
False, False, True, True, False, False, False, True, False, False, False, False,
False, False, True, False, False, True, True, True, True, False, True, True,
True, True, False])
estimator_3_features = np.array([False, True, False, False, True, False, True, True, True, True, False, True,
 True, True, True, True, True, True, False, True, False, True, True, True,
 True, False, False, True, False, False, False, False, False, True, True, True,
 False, False, False, False, True, True, True, False, True, True, True, False,
 False, True, True, True, False, True, False, True, True, True, False, True,
 False, True, True, True, True, False, False, True, True, False, False, False,
 False, True, False, False, False, False, True, True, True, False, True, True,
 True, True, True, True, False, False, False, False, False, True, False, False,
 True, False, True, True, False, True, False, True, True, False, False, True,
 False, True, False, False, False, False, False, False, True, False, True, False,
 True, True, False, True, True, False, True, True, False, True, False, True,
 False, False, False, True, False, True, True, True, True, True, False, True,
 True, True, False, False, True, False, False, False, True, False, True, False,
 False, True, False, False, False, False, True, False, True, False, False, True,
 False, True, False, False, True, True, False, False, False, False, False, True,
 False, False, True])
estimator_4_features = np.array([False, True, False, True, True, False, True, True, False, True, True, False,
 True, False, False, True, True, True, True, False, True, False, True, False,
 True, True, False, True, True, False, False, True, False, True, True, False,
 False, True, True, False, False, True, False, True, True, True, False, True,
 False, False, False, False, False, False, False, False, False, True, True, True,
 True, True, True, False, True, False, False, True, True, True, True, True,
 False, False, False, False, True, True, True, True, False, True, True, True,
 True, True, True, False, True, True, False, False, True, True, True, False,
 True, True, True, False, True, False, True, False, False, True, False, False,
 False, False, True, True, True, True, False, False, True, True, True, True,
 False, False, False, False, True, True, False, False, True, True, True, False,
 True, True, False, True, True, True, True, True, False, False, False, True,
 True, False, True, True, False, True, True, True, True, True, False, True,
 True, True, True, False, True, True, False, True, False, False, True, True,
 True, True, True, False, False, True, False, True, True, False, False, True,
 False, True, False])
okk = [estimator_0_features,estimator_1_features,estimator_2_features,estimator_3_features,estimator_4_features]

models_list = []
for i, partition in enumerate(X_train_non_druggable_partitions[:-1]):
    X_combined = np.concatenate((X_train_druggable, partition))
    y_combined = np.concatenate((np.ones(len(X_train_druggable)), np.zeros(len(partition))))

    clf = xgb.XGBClassifier(objective='binary:logistic', random_state=42)

    print(len(okk[i]))
    X_combined = X_combined[:, okk[i]]
    clf.fit(X_combined, y_combined)
    models_list.append(clf)

y_preds = []
for i,model in enumerate(models_list):
  y_pred = model.predict(np.asarray(X_test)[:, okk[i]])
  y_preds.append(y_pred)

majority_preds = np.mean(y_preds, axis=0)
majority_preds = np.round(majority_preds)

from sklearn.metrics import accuracy_score
accuracy_metrics = {}
for i, y_pred in enumerate(y_preds):
  accuracy_metrics[f"partition_{i}"]={
      "accuracy_total": accuracy_score(y_test, y_pred),
      "accuracy_druggable": accuracy_score(y_test[y_test == 1], y_pred[y_test == 1]),
      "accuracy_non_druggable": accuracy_score(y_test[y_test == 0], y_pred[y_test == 0]),
  }

accuracy_metrics["majority"] = {
    "accuracy_total": accuracy_score(y_test, majority_preds),
    "accuracy_druggable": accuracy_score(y_test[y_test == 1], majority_preds[y_test == 1]),
    "accuracy_non_druggable": accuracy_score(y_test[y_test == 0], majority_preds[y_test == 0]),
}

df = pd.DataFrame(accuracy_metrics).transpose()
df

183
183
183
183
183


,accuracy_total,accuracy_druggable,accuracy_non_druggable
partition_0,0.741667,0.731667,0.751667
partition_1,0.736667,0.740000,0.733333
partition_2,0.740000,0.743333,0.736667
partition_3,0.742500,0.750000,0.735000
partition_4,0.744167,0.755000,0.733333
majority,0.760833,0.761667,0.760000
